# 1. RNN
**: Recurrent Neural Network(순환 신경망)**
- 연속된 데이터가 순서대로 입력되었을 때 앞서 받은 데이터를 잠시 기억해 놓는 방법
- 기억된 데이터 당 중요도 가중치를 주면서 다음 데이터로 넘어감
- 학습 시 이전 데이터를 반영하여 보다 나은 weight와 baias를 계산할 수 있도록한다.

<img width="693" alt="image" src="https://user-images.githubusercontent.com/88031549/200717000-7910c40c-8719-49d0-98dd-f25212ff19bd.png">

---
# 2. LSTM  
: Long Short Term Memory
- RNN의 기울기 소실 문제 보완을 위해 기억 값의 가중치를 관리하는 방법
- 한 층에서 반복되기 직전에 다음 층으로 기억된 값을 넘길지 관리하는 단계를 하나 더 추가

<img width="578" alt="image" src="https://user-images.githubusercontent.com/88031549/200717128-5d73e269-8ae4-4fc5-be94-33b12ea402ec.png">

## 2-1. LSTM 종류
1 ) 다수 입력 단일 출력  
2 ) 단일 입력 다수 출력  
3 ) 다수 입력 다수 출력  
(*단일x단일은 x)

<img width="589" alt="image" src="https://user-images.githubusercontent.com/88031549/200717968-0bb164d6-6e17-4603-b6bd-23620c653842.png">

---
# 3. RNN 실습
- 입력된 문장의 의미를 파악하는 것 = 모든 단어를 종합하여 하나의 카테고리로 분류하는 작업

## 3-1. Data

In [28]:
import numpy as np
from keras.datasets import reuters # 로이터 뉴스 데이터 셋 불러오기

# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

# 불러온 데이터를 학습셋과 테스트셋으로 나누기
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

padding & categorical

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 짧은 문장에다 0을 채워넣어 가장 긴 문장과 길이를 맞춰 colum크기를 같게한다.
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

y_train = y_data[:1395]
y_test = y_data[1395:]

# 데이터의 모양 출력하기
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


## 3-2. modeling(RNN)

In [33]:
# 간단한 딥러닝
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, SimpleRNN

model = Sequential()
model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
model.add(SimpleRNN(50, return_sequences = False))
model.add(Dense(46, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [34]:
model.fit(X_train, y_train, epochs = 200, batch_size = 50)


Epoch 1/200
28/28 [==============================] - 1s 11ms/step - loss: 2.5332 - accuracy: 0.4946
Epoch 2/200
28/28 [==============================] - 0s 11ms/step - loss: 1.2427 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1836 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1614 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1487 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1240 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1192 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1159 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1034 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 0s 11ms/step - loss: 1.1050 - accuracy: 0.7147

## 3-3. Evaluation  
- 평가해주는 y와 y^에는 np.argmax 해줘야함

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

19/19 [==============================] - 1s 6ms/step
0.7929883138564274


---

# 4. LSTM 실습

In [41]:
# 단층 LSTM을 구현하기 위한 함수
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax')) # 이렇게 따로 활성화함수를 정의해줘도 된다.
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [42]:
# Modeling
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)
# verbose는 이러쿵저러쿵 과정을 설명하는 옵션임. print하는거 보기 싫으면 0으로!(verbose -> 수다쟁이라는 뜻.)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

/var/folders/x3/lny9bbwx64b561c29r4y4z400000gn/T/ipykernel_60587/1127062677.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)


Epoch 1/200


/Users/kyle/opt/anaconda3/envs/py310/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


28/28 [==============================] - 1s 13ms/step - loss: 3.0213 - accuracy: 0.5986
Epoch 2/200
28/28 [==============================] - 0s 15ms/step - loss: 1.3450 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 0s 14ms/step - loss: 1.1710 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 0s 13ms/step - loss: 1.1241 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 0s 14ms/step - loss: 1.0772 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 0s 13ms/step - loss: 1.0289 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 0s 13ms/step - loss: 0.9692 - accuracy: 0.7527
Epoch 8/200
28/28 [==============================] - 0s 12ms/step - loss: 0.9090 - accuracy: 0.7957
Epoch 9/200
28/28 [==============================] - 0s 13ms/step - loss: 0.8854 - accuracy: 0.7921
Epoch 10/200
28/28 [==============================] - 0s 13ms/step - loss: 0.8329 - accuracy: 0.8079
Epoch 11/20

> argmax는 가장 높은 숫자(확률)의 위치(=class번호)를 반환하는 함수.
> - predict를했을 때 y_pred는 각 클래스에 대한 확률값이 나온다. 여기서 argmax를 사용하면 가장 큰 확률의 위치를 반환하는데, 이 위치가 class 번호이다.

> KerasClassifier를 사용하면 predict를 할 때 y_pred에 argmax가 적용되기 때문에 위 code에서는 y^에 argmax를 사용x